In [182]:
import pandas as pd
import numpy as np

## Making the first,second,third and fourth columns of the dataframe as date,month,year and time

In [184]:
data=pd.DataFrame()
temp=pd.read_csv('../Data Sources/ENTSO-E/Load/Croatia.csv')
data['Day']=temp['Time (CET)'].str[:2]
data['Month']=temp['Time (CET)'].str[3:5]
data['Year']=temp['Time (CET)'].str[6:10]
data['Time']=temp['Time (CET)'].str[11:16]+' - '+temp['Time (CET)'].str[29:35]

### In the country speciic Load data set, Austria,Belgium,Germany,Hungary,Netherlands report data every 15 minutes. Therefore, these countries have 35044 data points per year. UK and Ireland report data every 30 minutes henece these countries have 17522 datapoints per year. All the others report every 1 hour hence have 8761 datapoints per year. In Genearion dataset, situation is same as abobe except Belgium reports hourly data hence have 8761 datapoints. In Transmission dataset, all countries report data hourly. Therefroe,it is easy if all the data are converted to hourly data. To do that in the countries with 35044 datapoints, mean is calculated in every successive 4 datapoints. In the countries with 17522 datapoints, mean is calculated in every successive 2 datapoints.`

In [185]:
def mean_calculator(df):
    length=len(df.index)
    if length==35044:
        divider=4
    elif length==17522:
        divider=2
    else:
        divider=1

    df=df.groupby(np.arange(length)//divider).mean()
    return(df)    

# Preparing Load Data

In [186]:
def load(countries):

    load_data=pd.DataFrame()
    load_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/Load/{country}.csv')
        temp=mean_calculator(temp)
        load_data[f'{abbr}']=temp[f'Actual Total Load [MW] - {country} ({abbr})']
        
    return(load_data)

# Preparing Generation Data

In [187]:
def generation(countries):
    
    generation_data=pd.DataFrame()
    generation_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/Generation/{country}.csv',low_memory=False)
        temp=mean_calculator(temp)

        fuels=np.array(temp.columns)
        for fuel in fuels:
            edited_fuel=fuel[:-26]
            generation_data[f'{abbr} - {edited_fuel}']=temp[fuel]

    return(generation_data)